In [1]:
#importowanie bibliotek
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [1]:
#wczytanie danych
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
y = train_data.Survived

train_data.describe()


NameError: name 'pd' is not defined

In [5]:
#okreslenie cech dla naszego modelu
features = ["Pclass", "Sex", "Age", "Parch", "Fare", "SibSp"]

X = train_data[features]

#stworzenie enkodera zamieniajacego zmienne opisowe na liczbowe
ordinal_encoder = OrdinalEncoder()

encoder_cols = ["Sex"]

#podzial danych na treningowe i walidacyjne
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

train_X[encoder_cols] = ordinal_encoder.fit_transform(train_X[encoder_cols])
val_X[encoder_cols] = ordinal_encoder.transform(val_X[encoder_cols])

#stworzenie imputera ktory uzupelnia brakujace wartosci
imputer = SimpleImputer()

imputer_cols = ["Age","Fare"]

train_X[imputer_cols] = imputer.fit_transform(train_X[imputer_cols])
val_X[imputer_cols] = imputer.transform(val_X[imputer_cols])

,Pclass,Sex,Age,Parch,Fare,SibSp
count,668.000000,668.000000,668.000000,668.000000,668.000000,668.000000
mean,2.321856,0.651198,30.073682,0.383234,31.571556,0.498503
std,0.830275,0.476948,13.244688,0.787736,46.017085,1.019303
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,22.000000,0.000000,7.895800,0.000000
50%,3.000000,1.000000,30.073682,0.000000,14.454200,0.000000
75%,3.000000,1.000000,36.000000,0.000000,30.500000,1.000000
max,3.000000,1.000000,80.000000,6.000000,512.329200,8.000000


In [6]:
#stworzenie modelu, przetrenowanie i predykcja
rf_model = RandomForestClassifier(n_estimators=500,random_state=1)
rf_model.fit(train_X, train_y)
rf_val_predictions = rf_model.predict(val_X)
rf_val_f1 = f1_score(val_y,rf_val_predictions)

print("Validation F1 Score for Random Forest Model: {}".format(rf_val_f1))

Validation F1 Score for Random Forest Model: 0.703030303030303


In [7]:
#obrobka danych testowych do wyslania na konkurs kaggle
test_X = test_data[features].copy()
test_X[encoder_cols] = ordinal_encoder.transform(test_X[encoder_cols])
test_X[imputer_cols] = imputer.transform(test_X[imputer_cols])

In [8]:
#dokonanie predykcji i stworzenie pliku csv do wyslania
results = rf_model.predict(test_X)
((pd.DataFrame(results)).set_index(test_data['PassengerId'])).to_csv("predictions.csv",header=['Survived'],index_label='PassengerId')

numpy.ndarray